In [1]:
''' PANDAS AND NUMPY
 KUBITA MATEUSZ
'''

' PANDAS AND NUMPY\n KUBITA MATEUSZ\n'

In [3]:
#Ad. 1 Wczytanie ramek danych, bibliotek, stworzenie bazy danych i załączenie tam ramek danych

#wczytanie bibliotek
import pandas as pd
import numpy as np
import os, os.path
import sqlite3

#Wczytanie ramek danych
Badges = pd.read_csv("frames//Badges.csv.gz", comment = "#", compression = "gzip")
Comments = pd.read_csv("frames//Comments.csv.gz", comment = "#", compression = "gzip")
PostLinks = pd.read_csv("frames//PostLinks.csv.gz", comment = "#", compression = "gzip")
Posts = pd.read_csv("frames//Posts.csv.gz", comment = "#", compression = "gzip")
Tags = pd.read_csv("frames//Tags.csv.gz", comment = "#", compression = "gzip")
Votes = pd.read_csv("frames//Votes.csv.gz", comment = "#", compression = "gzip")
Users = pd.read_csv("frames//Users.csv.gz", compression = "gzip")

#Stworzenie bazy danych i wczytanie tam ramek danych
data_base = os.path.join("data_base8")
conn = sqlite3.connect(data_base)

Badges.to_sql("Badges", conn)
Comments.to_sql("Comments", conn)
PostLinks.to_sql("PostLinks", conn)
Posts.to_sql("Posts", conn)
Tags.to_sql("Tags", conn)
Votes.to_sql("Votes", conn)
Users.to_sql("Users", conn)

89120

In [4]:
#ZAPYTANIE 1
sql1 = pd.read_sql_query("""SELECT Count, TagName
FROM Tags
WHERE Count > 1000
ORDER BY Count DESC
""",conn)

In [5]:
res1 = Tags[Tags.Count > 1000].reset_index(drop = True)[["Count", "TagName"]].sort_values(["Count"], ascending = [False]).reset_index(drop = True)

In [6]:
#POPRAWNOSC WYNIKOW 1
res1.equals(sql1)

True

In [7]:
#ZAPYTANIE 2
sql2 = pd.read_sql_query("""SELECT Location, COUNT(*) AS Count
FROM (
SELECT Posts.OwnerUserId, Users.Id, Users.Location
FROM Users
JOIN Posts ON Users.Id = Posts.OwnerUserId
)
WHERE Location NOT IN ('')
GROUP BY Location
ORDER BY Count DESC
LIMIT 10""", conn)

In [8]:
res2 = Posts.merge(Users, left_on="OwnerUserId", right_on = "Id")
res2 = res2[["OwnerUserId", "Id_x", "Location"]]
res2 = pd.DataFrame(res2.Location.value_counts()).reset_index()
res2.columns = ["Location", "Count"]
res2 = res2.iloc[0:10]

In [9]:
#POPRAWNOSC WYNIKOW 2
res2.equals(sql2)

True

In [10]:
#ZAPYTANIE 3
sql3 = pd.read_sql_query("""SELECT Year, SUM(Number) AS TotalNumber
FROM (
SELECT
Name,
COUNT(*) AS Number,
STRFTIME('%Y', Badges.Date) AS Year
FROM Badges
WHERE Class = 1
GROUP BY Name, Year
)
GROUP BY Year
ORDER BY TotalNumber
""",conn)

In [11]:
#AD.1
tmp = Badges.copy()
tmp["Date"] = tmp["Date"].str.slice(0,4) #wybranie podzbioru stringa od 0 do 4 czyli rok w dacie
res3 = tmp[tmp.Class == 1]
#AD.2
res3 = res3.groupby(["Name", "Date"]).size().reset_index()
res3.columns = ["Name", "Year", "Count"]
res3 = res3[["Count"]].groupby(res3.Year).apply(np.sum).reset_index()
res3.columns = ["Year", "TotalNumber"]
res3.sort_values(by = ["TotalNumber"], inplace = True)
res3.reset_index(drop = True, inplace = True)

In [12]:
#POPRAWNOSC WYNIKOW 3
res3.equals(sql3)

True

In [13]:
#ZAPYTANIE 4
sql4 = pd.read_sql_query("""SELECT
Users.AccountId,
Users.DisplayName,
Users.Location,
AVG(PostAuth.AnswersCount) as AverageAnswersCount
FROM
(
SELECT
AnsCount.AnswersCount,
Posts.Id,
Posts.OwnerUserId
FROM (
SELECT Posts.ParentId, COUNT(*) AS AnswersCount
FROM Posts
WHERE Posts.PostTypeId = 2
GROUP BY Posts.ParentId
) AS AnsCount
JOIN Posts ON Posts.Id = AnsCount.ParentId
) AS PostAuth
JOIN Users ON Users.AccountId=PostAuth.OwnerUserId
GROUP BY OwnerUserId
ORDER BY AverageAnswersCount DESC, AccountId ASC
LIMIT 10
""",conn)

In [14]:
#AD.1 Stworzenie AnsCount
AnsCount = Posts[Posts.PostTypeId == 2]
AnsCount = pd.DataFrame(AnsCount.groupby("ParentId").size().reset_index())
AnsCount.columns = ["ParentId", "AnswersCount"]
#AD.2 Stworzenie PostAuth
PostAuth = Posts.merge(AnsCount, left_on = "Id", right_on = "ParentId")
PostAuth = PostAuth[["AnswersCount","Id", "OwnerUserId"]]
#AD.3
res4 = Users.merge(PostAuth, left_on = "AccountId", right_on = "OwnerUserId")
res4 = res4.groupby(["AccountId","DisplayName", "Location"], dropna = False)["AnswersCount"].agg(np.mean).reset_index()
res4.columns = ["AccountId", "DisplayName", "Location", "AverageAnswersCount"]
res4.sort_values(["AverageAnswersCount", "AccountId"], ascending = [False,True], inplace = True)
res4 = res4.iloc[0:10]
res4.reset_index(drop = True, inplace = True)

In [15]:
#POPRAWNOSC WYNIKOW 4
res4.equals(sql4)

True

In [16]:
#ZAPYTANIE 5
sql5 = pd.read_sql_query("""SELECT Posts.Title, Posts.Id,
STRFTIME('%Y-%m-%d', Posts.CreationDate) AS Date,
VotesByAge.Votes
FROM Posts
JOIN (
SELECT
PostId,
MAX(CASE WHEN VoteDate = 'new' THEN Total ELSE 0 END) NewVotes,
MAX(CASE WHEN VoteDate = 'old' THEN Total ELSE 0 END) OldVotes,
SUM(Total) AS Votes
FROM (
SELECT
PostId,
CASE STRFTIME('%Y', CreationDate)
WHEN '2021' THEN 'new'
WHEN '2020' THEN 'new'
ELSE 'old'
END VoteDate,
COUNT(*) AS Total
FROM Votes
WHERE VoteTypeId IN (1, 2, 5)
GROUP BY PostId, VoteDate
) AS VotesDates
GROUP BY VotesDates.PostId
HAVING NewVotes > OldVotes
) AS VotesByAge ON Posts.Id = VotesByAge.PostId
WHERE Title NOT IN ('')
ORDER BY Votes DESC
LIMIT 10
""", conn)

In [17]:
pd.options.mode.chained_assignment = None
#AD.1 Stworznie VotesDates
VotesDates = Votes[(Votes.VoteTypeId == 1) | (Votes.VoteTypeId == 2) | (Votes.VoteTypeId == 5)]
VotesDates.loc[:,"CreationDate"] = VotesDates.loc[:,"CreationDate"].str.slice(0,4)
VotesDates["VoteDate"] = np.where((VotesDates["CreationDate"] == "2021") | (VotesDates["CreationDate"] == "2020"),"new","old")
VotesDates = VotesDates.groupby(["PostId","VoteDate"]).size().reset_index()
VotesDates.columns = ["PostId", "VoteDate", "Total"]
#AD.2 Podwojne mergowanie aby zmodyfikowac ksztalt ramki danych 
VotesDates1 = VotesDates[VotesDates.VoteDate == "new"][["PostId","Total"]].reset_index(drop = True)
VotesDates2 = VotesDates[VotesDates.VoteDate == "old"][["PostId","Total"]].reset_index(drop = True)
VotesDates3 = pd.DataFrame(VotesDates.PostId.unique())
VotesDates3.columns = ["PostId"]
resA = VotesDates3.merge(VotesDates1, left_on = "PostId", right_on = "PostId", how = "outer")
resB = resA.merge(VotesDates2,how ="outer", on = "PostId")
resB.fillna(0, inplace = True)
resB.columns = ["PostId","NewVotes", "OldVotes"]
resB["Votes"] = resB["NewVotes"] + resB["OldVotes"]
resB = resB[resB.NewVotes > resB.OldVotes].reset_index(drop = True)
#AD.3
res5 = Posts.merge(resB, left_on = "Id", right_on = "PostId")
res5["Date"] = res5["CreationDate"].str.slice(0,10)
res5 = res5[["Title", "Id", "Date", "Votes"]]
res5.dropna(subset = ["Title"], inplace = True)
res5.sort_values(by = ["Votes", "Id"], ascending = [False, True], inplace = True)
res5 = res5.iloc[0:10]
res5.reset_index(drop = True,inplace = True)
res5["Votes"] = res5["Votes"].astype("int64")

In [18]:
#POPRAWNOSC WYNIKOW 5
res5.equals(sql5)

True

In [20]:
conn.close()